In [3]:
# First cell - Load environment variables from specific path
from dotenv import load_dotenv
import os

# Specify the exact path to your .env file
ENV_PATH = r"./ragtest/.env"

# Load .env file from specific path
if not os.path.exists(ENV_PATH):
    raise FileNotFoundError(f".env file not found at {ENV_PATH}")

load_dotenv(ENV_PATH)

# Verify API key loaded
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in .env file")

print(f"Environment variables loaded successfully from {ENV_PATH}!")


Environment variables loaded successfully from ./ragtest/.env!


In [5]:


# Second cell - Import required libraries
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from tqdm import tqdm

# Define constants
INPUT_DIR = "./ragtest/input"
INDEX_DIR = "./ragtest/faiss_index"
CACHE_DIR = "./cache"

# Create necessary directories
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(os.path.dirname(INDEX_DIR), exist_ok=True)

# Third cell - Create or load index
def create_or_load_index():
    if os.path.exists(INDEX_DIR):
        print("Loading existing index...")
        embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small"
        )
        vector_store = FAISS.load_local(INDEX_DIR, embeddings, allow_dangerous_deserialization=True)
        return vector_store
    
    print("Creating new index...")
    loader = DirectoryLoader(
        INPUT_DIR,
        glob="**/*.txt",
        loader_cls=TextLoader,
        loader_kwargs={'autodetect_encoding': True}
    )
    documents = loader.load()
    print(f"Loaded {len(documents)} documents")
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=100,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    splits = text_splitter.split_documents(documents)
    print(f"Split into {len(splits)} chunks")
    
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",
    )
    
    print("Creating vector store...")
    vector_store = FAISS.from_documents(splits, embeddings)
    vector_store.save_local(INDEX_DIR)
    print(f"Index saved to {INDEX_DIR}")
    
    return vector_store

# Create or load the index
vector_store = create_or_load_index()

# Fourth cell - Setup QA chain
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

Context: {context}

Question: {question}

Answer: Let me answer based on the specific context provided."""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Initialize LLM and QA chain
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(
        search_kwargs={
            "k": 3,
            "fetch_k": 5
        }
    ),
    chain_type="stuff",
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": True
    }
)

# Fifth cell - Helper function to get document stats
def get_document_stats():
    doc_count = len(vector_store.docstore._dict)
    print(f"\nIndex Statistics:")
    print(f"Total number of chunks in index: {doc_count}")
    
    sample_ids = list(vector_store.docstore._dict.keys())[:5]
    print(f"\nSample of first few document chunks (IDs):")
    for id in sample_ids:
        print(f"- {id}")

# Show initial stats
get_document_stats()

# Sixth cell - Function to ask a single question
def ask_question(question: str):
    try:
        result = qa_chain.invoke(question)
        return result['result']
    except Exception as e:
        return f"Error processing question: {str(e)}"



Loading existing index...

Index Statistics:
Total number of chunks in index: 307

Sample of first few document chunks (IDs):
- 41af3c29-02ac-4d97-89b2-6c53b6c72fc4
- e0abd438-468f-4258-8081-785a14210599
- 50515279-8b79-4568-b83a-0bf4a5f8b640
- ee357d3e-45ac-46c9-ac84-4349e0d4bb1f
- e66d7bab-4a37-4bbe-801b-2b47b5729be0


In [15]:
# Example usage:
question = "What topics are covered in these podcasts?"
question = "Which episodes deal primarily with tech policy and government regulation?"
question = "How do guests perceive the impact of privacy laws on technology development?"
# question = "Do any guests discuss the balance between innovation and ethical considerations?"
question = "Are collaborations between tech companies and governments discussed and how?"
answer = ask_question(question)


print(answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

Context: KEVIN SCOTT: Which is fantastic. And I'm really, really glad that you are engaged here, because one of the real difficulties that I see is that these issues are, I think, unprecedented in terms of the complexity. There's just nothing in human experience that would let you develop an intuitive feel for what's happening under the hood of some of this technology. 
And it's not all sinister, right? I would put forth that most of this stuff, like the vast majority, especially built by tech companies, is like well-intentioned technology, where we haven't thought about the second-order consequences of what people are going to do with them, and a variety of different things. 
But getting the public to be informed enoug

In [9]:
answer = ask_question("What are the episodes that talk about Billgates?")
print(answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

Context: Behind the Tech with Kevin Scott Podcast
Episode 52 – Bill Gates

BILL GATES: It was stunning, it was mind blowing. After the biology question I had them type in, “What do you say to a father with a sick child?” And it gave this very careful, excellent answer that was perhaps better than any of us in the room might have given. And so, it was like, wow, what is it – what is the scope of this thing? Because this is way better.
[MUSIC]

KEVIN SCOTT: Hi, everyone. Welcome to Behind the Tech. I’m your host, Kevin Scott, Chief Technology Officer for Microsoft. 

In this podcast, we’re going to get behind the tech. We’ll talk with some of the people who have made our modern tech world possible and understand what moti

In [16]:
# # "What topics are covered in these podcasts?"
# #rag answere
# # > Finished chain.
# # The podcasts cover topics related to the history of computing, insights into modern technology, the motivations behind technological innovations, and the experiences of influential figures in the tech industry. They aim to inspire listeners, particularly developers, by providing a deeper understanding of the tech world and encouraging reflection on the values of growth and humanity in technology.
# # graph rag answer
# # SUCCESS: Global Search Response:
# # The podcasts cover a range of significant topics, primarily focusing on the intersection of technology, artificial intelligence (AI), and their societal implications. Below are the key themes discussed:

# # ### 1. **Artificial Intelligence and Its Societal Impact**
# # A central theme in the podcasts is the transformative potential of AI across various sectors, including healthcare, education, and public health. Discussions emphasize the ethical considerations surrounding AI development and the importance of responsible practices to mitigate potential negative impacts on society [Data: Reports (46, 52, 14, 20, 24, +more); (41, 43, 28, 49, 55, +more)].

# # ### 2. **Mentorship and Role Models in Technology**
# # The podcasts highlight the significance of mentorship and the influence of role models in inspiring future technology leaders. Figures like Kevin Scott advocate for the importance of role models in encouraging youth to pursue careers in STEM fields, thereby addressing the need for a diverse and skilled workforce [Data: Reports (19, 20, 14, 24, 25, +more)].

# # ### 3. **Educational Equity and Access to Technology**
# # Another prominent topic is the role of technology in bridging economic disparities, particularly in underserved and rural communities. The discussions focus on initiatives aimed at improving access to technology and education, which are crucial for fostering equity in educational opportunities [Data: Reports (43, 12, 7, 49, 28, +more)].        

# # ### 4. **Intersection of Art and Technology**
# # The podcasts explore how technology is reshaping creative industries, particularly in animation and music. Insights from industry leaders illustrate the collaborative potential of AI in augmenting human creativity rather than replacing it, showcasing innovative approaches to music-making and artistic expression [Data: Reports (51, 50, 45, 14, 20, +more); (37, 39, 41, 49, 28, +more)].

# # ### 5. **Sustainable Practices in Technology Development**
# # Sustainability is a recurring theme, with discussions emphasizing the need for minimizing environmental impact and energy consumption in the development of AI and robotics. This focus on sustainable practices is essential for ensuring that technological advancements do not come at the expense of the environment [Data: Reports (55, 41, 28, 12, 49, +more)].

# # ### Conclusion
# # Overall, the podcasts provide a rich exploration of how technology, particularly AI, intersects with various societal issues, emphasizing the need for responsible development, mentorship, and equitable access to resources. These discussions are vital for understanding the future landscape of technology and its implications for society.


# # Question How do guests perceive the impact of privacy laws on technology development?
# # rag answer
# # The guests perceive the impact of privacy laws on technology development as a necessary consideration that can help mitigate risks to privacy while also enabling advancements in technology. They acknowledge that regulation can play a role in addressing privacy concerns, but they also highlight the potential for technological breakthroughs, such as homomorphic encryption, to facilitate the use of data without compromising privacy. Overall, they see a need for a balance between regulation and technological innovation to ensure safe and responsible deployment of technology.
# # Graph ragh answer
# # SUCCESS: Global Search Response:
# # ### Overview of Guest Perceptions on Privacy Laws and Technology Development

# # The intersection of privacy laws and technology development is a critical area of concern, particularly as advancements in fields like artificial intelligence (AI) continue to evolve. Guests have expressed a range of perceptions regarding how privacy regulations influence technological innovation, especially in sensitive sectors such as healthcare.

# # ### Privacy Concerns and Technological Advancement

# # Guests have noted that the integration of AI into healthcare raises significant privacy concerns. The collection and utilization of personal health data for AI systems may lead to potential misuse of this sensitive information. This relationship underscores the necessity for stringent regulations and ethical guidelines to protect patient information while still allowing for technological advancements in healthcare [Data: Reports (42)].

# # Moreover, as healthcare increasingly relies on data-driven technologies, the potential for misuse of personal health information becomes a pressing issue. Addressing these privacy concerns is essential for maintaining public trust and ensuring the responsible use of AI in sensitive areas like healthcare [Data: Reports (42)]. Guests believe that without adequate privacy protections, the public may become wary of technological innovations, which could stifle development and adoption.

# # ### The Balance Between Innovation and Regulation

# # Guests have emphasized the importance of finding a balance between fostering innovation and implementing necessary privacy regulations. They argue that while privacy laws are crucial for protecting individuals, overly stringent regulations may hinder technological progress. This tension suggests that policymakers must carefully consider how to craft laws that safeguard privacy without impeding the growth of beneficial technologies.

# # In conclusion, guests perceive privacy laws as a double-edged sword in the realm of technology development. While they recognize the importance of protecting personal information, they also express concerns that excessive regulation may limit the potential benefits that new technologies can offer, particularly in critical sectors like healthcare. The ongoing dialogue around this issue will likely shape the future landscape of technology and privacy.


# # Question : Do any guests discuss the balance between innovation and ethical considerations?"

# # > Finished chain.
# # Yes, the guests discuss the balance between innovation and ethical considerations. Judy Estrin mentions the need for the industry to understand the consequences of their actions and to embrace opportunities that come with ethical considerations, similar to how other industries have had to adapt. She emphasizes the importance of being aware of the impacts of technology on democracy, humanity, and media, highlighting the trade-offs that come with technological progress.

# # GRAPH RAG ANSWER
# # SUCCESS: Global Search Response:
# # ## Discussion on Innovation and Ethical Considerations

# # Several discussions highlight the critical balance between innovation in artificial intelligence (AI) and the ethical considerations that accompany such advancements. This discourse is essential for ensuring that technological progress does not come at the expense of societal values and responsibilities.

# # ### Key Perspectives

# # 1. **Sustainable AI Development**
# #    Kevin Scott emphasizes the necessity for sustainable practices in AI development, particularly concerning the energy costs associated with advanced models like GPT-3. His conversations with experts underline the importance of balancing innovation with ethical considerations, advocating for responsible and equitable development of AI technologies [Data: Reports (43, 41, +more)].

# # 2. **Societal Implications of Technology**
# #    Judy Estrin raises significant concerns regarding the societal implications of technology, including data privacy, disinformation, and election hacking. She stresses the need for ethical considerations in technology development, warning of the potential consequences of neglecting these issues. This highlights the importance of balancing innovation with ethical responsibilities to mitigate risks associated with technological advancements [Data: Reports (20, 24, 32)].

# # 3. **Ethical Implications of AI**
# #    The community surrounding Greg Shaw and Kevin Scott discusses the ethical implications of AI, particularly its impact on society and human intelligence. This dialogue is crucial for understanding how innovation in AI must be balanced with ethical considerations to ensure responsible development and application of technology [Data: Reports (44, 40)].

# # 4. **Artificial General Intelligence (AGI)**
# #    The pursuit of Artificial General Intelligence (AGI) raises significant ethical questions regarding its societal impacts. Discussions around AGI emphasize the need for careful consideration of the ethical ramifications that accompany advancements in AI, indicating a recognition of the balance required between innovation and ethical responsibility [Data: Reports (40)].

# # 5. **Responsible Use of AI Technologies**
# #    Bill Gates has been actively involved in discussions surrounding AI, particularly its implications for various industries. His collaboration with organizations like OpenAI reflects a focus on the responsible use of AI technologies, indicating that ethical considerations are paramount alongside technological advancements [Data: Reports (52, 29)].

# # ### Conclusion

# # The discussions surrounding the balance between innovation and ethical considerations in AI development are multifaceted and critical. Stakeholders from various sectors emphasize the need for responsible practices that prioritize ethical implications, ensuring that technological advancements contribute positively to society while mitigating potential risks. This ongoing dialogue shall be essential as AI continues to evolve and integrate into various aspects of life.

# # Question  Are collaborations between tech companies and governments discussed and how?
# # RAG answer
# # > Finished chain.
# # Yes, collaborations between tech companies and governments are discussed in the context of addressing the impacts of technology on democracy, humanity, and media. Judy Estrin mentions the need for data to understand these impacts, highlighting that regulation can help manage privacy risks associated with data usage. Additionally, she emphasizes the potential for technological advancements, such as homomorphic encryption, to facilitate safe and responsible deployment of technology while allowing organizations to use data without compromising privacy. This suggests a collaborative approach where technology and policy work together to address complex challenges.
# # GRAPH RAG ANSWER
# # SUCCESS: Global Search Response:
# # ## Overview of Collaborations Between Tech Companies and Governments

# # The dataset reveals significant insights into the collaborations between tech companies and governments, particularly in the context of addressing societal challenges and advancing technology. These collaborations are often driven by influential figures in the tech industry and are evident in various initiatives aimed at improving education, healthcare, and space exploration.

# # ### Key Collaborations and Initiatives

# # 1. **100 Year Starship Initiative**:
# #    The partnership between the 100 Year Starship initiative and DARPA highlights a crucial collaboration between government entities and tech initiatives focused on interstellar flight. This relationship underscores the importance of government and military research in fostering innovative solutions for space exploration, indicating a strong alignment between technological advancements and governmental support [Data: Reports (23, +more)].

# # 2. **Influence of Key Figures**:
# #    The professional relationships of influential figures such as Greg Shaw and Bill Gates illustrate the potential for impactful collaborations. Their work at Microsoft and the Bill and Melinda Gates Foundation emphasizes the connection between technology and societal issues, particularly in education. This suggests that tech companies may engage with governmental bodies to address critical challenges through technology-driven initiatives [Data: Reports (44, +more)].

# # 3. **Educational Initiatives**:
# #    The focus on educational programs, especially in rural areas, indicates that tech companies like Microsoft may collaborate with local governments to enhance educational opportunities. Initiatives such as Breakthrough Silicon Valley aim to address educational equity, which may involve partnerships with governmental entities to tackle disparities in education [Data: Reports (12, 7)].

# # ### Implications for Future Collaborations

# # The data suggests that there is a growing need for collaboration between tech companies and governments, particularly in sectors like healthcare and education. The potential of AI in these fields may drive further partnerships aimed at improving public policy and addressing societal needs [Data: Reports (42, 29)]. Additionally, discussions among key figures in the tech industry regarding AI advancements may influence strategic directions that align with governmental policies, indicating a broader context of collaboration that could shape future initiatives [Data: Reports (28, 30, 54)].

# # ### Conclusion

# # In summary, the dataset indicates that collaborations between tech companies and governments are not only discussed but are also actively pursued through various initiatives. These partnerships are essential for addressing societal challenges and advancing technology, particularly in education and healthcare. The involvement of influential figures in these discussions may further enhance the potential for impactful collaborations in the future.


# # LOCAL QUERY

# "What are the episodes that talk about Billgates?"
# # RAG answer

#  Finished chain.
# # The context provided mentions Episode 52 of the "Behind the Tech with Kevin Scott" podcast, which features Bill Gates as a guest. This episode discusses various topics, including Gates' experiences with technology and his insights on AI.

# # GRAPH RAG RESPONSE

# # SUCCESS: Local Search Response:
# ### Episodes Featuring Bill Gates

# # Bill Gates has been featured in various discussions and podcasts that delve into his insights on technology, philanthropy, and artificial intelligence. Here are some notable episodes that highlight his contributions and thoughts:

# # 1. **Behind the Tech with Kevin Scott - Episode 52**
# #    - In this episode, Bill Gates discusses the advancements in artificial intelligence, particularly focusing on GPT-4 and its implications for technology and society. He shares his experiences and insights from a dinner where GPT-4 was first unveiled, emphasizing the transformative potential of AI technologies. Gates reflects on the evolution of AI and its impact on various industries, including education and public health [Data: Sources (28)].

# # 2. **Gates Notes**
# #    - Bill Gates shares his reading lists and thoughts on various topics through Gates Notes, which often includes discussions about technology and its societal implications. While not a traditional episode, this platform serves as a medium for Gates to express his views and engage with a broader audience [Data: Entities (219)].

# # 3. **Podcast Appearances**
# #    - Gates has made guest appearances on various podcasts, including discussions about his philanthropic efforts through the Gates Foundation and his views on educational technology. These appearances often highlight his commitment to addressing global challenges in health and education [Data: Relationships (300)].

# # 4. **Dinner Discussions**
# #    - Gates hosted a dinner where significant discussions about AI advancements took place, including the unveiling of GPT-4. This event brought together key figures in technology and showcased Gates' role in shaping the conversation around AI [Data: Relationships (279)].

# # 5. **Interviews and Collaborations**
# #    - Gates has collaborated with various individuals in the tech industry, including Greg Shaw, where they discuss the implications of AI and its potential to transform education and public health systems. These discussions often reflect Gates' vision for leveraging technology to improve lives globally [Data: Relationships (301)].

# # ### Commentary

# # Bill Gates remains a pivotal figure in the technology sector, and his insights continue to influence discussions around AI, education, and global health. His appearances in podcasts and discussions not only highlight his expertise but also his commitment to using technology as a tool for positive change. Through platforms like Gates Notes and various podcast episodes, Gates engages with audiences on critical issues, making his thoughts accessible to a wider community.

# # For those interested in the intersection of technology and philanthropy, these episodes and discussions provide valuable perspectives on how innovations can address some of the world's most pressing challenges.

SyntaxError: invalid syntax (2425739713.py, line 116)